In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import numpy as np
import cv2
import librosa
import urllib.request

# Download pre-trained haarcascade files
urllib.request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml", "haarcascade_frontalface_default.xml")
urllib.request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_fullbody.xml", "haarcascade_fullbody.xml")

# Function to extract pitch, intensity, and speech rate from audio
def extract_audio_features(audio_path):
    audio, sr = librosa.load(audio_path, sr=None)
    pitch = librosa.yin(audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    intensity = librosa.feature.rms(y=audio)
    speech_rate = np.count_nonzero(librosa.effects.split(audio)) / (len(audio) / sr)
    return pitch, intensity, speech_rate

# Function to analyze body language using OpenCV
def analyze_body_language(video_path):
    # Load pre-trained body language detection model
    body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
    # Load video
    cap = cv2.VideoCapture(video_path)
    # Initialize counters
    total_frames = 0
    body_detected_frames = 0
    # Loop through video frames
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detect bodies in the frame
            bodies = body_cascade.detectMultiScale(gray, 1.1, 4)
            if len(bodies) > 0:
                body_detected_frames += 1
            total_frames += 1
        else:
            break
    # Prevent ZeroDivisionError
    body_language_score = body_detected_frames / total_frames if total_frames != 0 else 0
    cap.release()
    return body_language_score

# Define weights for each factor
weight_pitch = 0.3
weight_intensity = 0.3
weight_speech_rate = 0.2
weight_body_language = 0.2

# Example usage
audio_path = "/content/drive/MyDrive/newaudio.wav" # Update with your audio file path
video_path = "/content/drive/MyDrive/file.mp4" # Update with your video file path

pitch, intensity, speech_rate = extract_audio_features(audio_path)
body_language_score = analyze_body_language(video_path)

# Calculate overall score
overall_score = (weight_pitch * np.mean(pitch) +
                 weight_intensity * np.mean(intensity) +
                 weight_speech_rate * speech_rate +
                 weight_body_language * body_language_score)

print("Confidence:", overall_score)




Confidence: 69.05148623100615
